In [42]:
import pandas as pd
import json
from environs import Env
from graph_functions import query_to_df,start_graph_session, start_graph_session_crashdown_w_auth
from IPython.display import Image
import numpy as np

env = Env()
#session = start_graph_session(env) 
session = start_graph_session_crashdown_w_auth(env) 

38687


## Workflow
### Initial exploration
1. MR-eve results: find interesting exposure
2. Manual MR: confirm the effect is there
3. Broad review of literature triples connection (melodi-presto style)
4. Traditional literature search with 'breast cancer'

### Deeper dive into the role/mechanism

5. Find related pathways (manual reactome search)
    - compare with literature triples
6. Pathway -> proteins -> genes
7. GWAS -> tophits -> genes
8. Compare gene overlap 6) and 7)

9. Check tissue exprossion for genes from 7)

### next part

??
link QTL?



In [43]:
# MR results
query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in ['ieu-a-1126','ieu-a-1127','ieu-a-1128']
AND  exposure.id  = 'met-a-362'
with mr, exposure, outcome
ORDER BY exposure.trait
RETURN exposure.id, exposure.trait, 
        outcome.id, 
        mr.pval, mr.b, mr.se, mr.method, mr.moescore
"""
df = query_to_df(session, query, True)

df['or'] = np.exp(df['mr.b'])
df['or_loci'] = np.exp(df['mr.b'] - 1.96 * df['mr.se'])
df['or_upci'] = np.exp(df['mr.b'] + 1.96 * df['mr.se'])

df


MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in ['ieu-a-1126','ieu-a-1127','ieu-a-1128']
AND  exposure.id  = 'met-a-362'
with mr, exposure, outcome
ORDER BY exposure.trait
RETURN exposure.id, exposure.trait, 
        outcome.id, 
        mr.pval, mr.b, mr.se, mr.method, mr.moescore



,exposure.id,exposure.trait,outcome.id,mr.pval,mr.b,mr.se,mr.method,mr.moescore,or,or_loci,or_upci
0,met-a-362,Betaine,ieu-a-1128,0.003013,-1.150799,0.387943,FE IVW,1.0,0.316384,0.147908,0.676765
1,met-a-362,Betaine,ieu-a-1127,0.300013,-0.173489,0.167395,FE IVW,1.0,0.840726,0.605571,1.167197
2,met-a-362,Betaine,ieu-a-1126,0.058986,-0.398726,0.211157,FE IVW,1.0,0.671174,0.443706,1.015256


In [ ]:
# connection with literature

# see R

In [44]:
# betaine GWAS -> pathways/expression data
query = """
MATCH (g:Gwas)-[tophits:OPENGWAS_TOPHITS]->(v:Variant)-[]->(gene:Gene)
where g.id = 'met-a-362'
return g.id, v.name, gene.name
"""
df = query_to_df(session, query, True)
df


MATCH (g:Gwas)-[tophits:OPENGWAS_TOPHITS]->(v:Variant)-[]->(gene:Gene)
where g.id = 'met-a-362'
return g.id, v.name, gene.name



,g.id,v.name,gene.name
0,met-a-362,rs2851391,CBS
1,met-a-362,rs2851391,DGKG
2,met-a-362,rs2851391,ARHGEF11
3,met-a-362,rs2851391,ZNF687
4,met-a-362,rs715,CPS1
5,met-a-362,rs499368,SLC6A12
6,met-a-362,rs499368,SLC6A12
7,met-a-362,rs16876394,DMGDH


In [45]:
# https://reactome.org/content/detail/R-HSA-6798163
query = """
MATCH (n:Pathway)-[]-(p:Protein)-[]-(g:Gene)
where n.id in  ['R-HSA-6798163', 'R-HSA-442660']
return n.name, p.name, g.name
"""
df = query_to_df(session, query, True)
df



MATCH (n:Pathway)-[]-(p:Protein)-[]-(g:Gene)
where n.id in  ['R-HSA-6798163', 'R-HSA-442660']
return n.name, p.name, g.name



,n.name,p.name,g.name
0,Choline catabolism,Q93088,BHMT
1,Choline catabolism,Q8WWI5,SLC44A1
2,Choline catabolism,Q8NE62,CHDH
3,Choline catabolism,P49419,ALDH7A1
4,Choline catabolism,Q9UL12,SARDH
5,Choline catabolism,Q9UI17,DMGDH
6,Na+/Cl- dependent neurotransmitter transporters,Q96N87,SLC6A18
7,Na+/Cl- dependent neurotransmitter transporters,Q695T7,SLC6A19
8,Na+/Cl- dependent neurotransmitter transporters,Q05940,SLC18A2
9,Na+/Cl- dependent neurotransmitter transporters,Q01959,SLC6A3


In [30]:
# check tissue expression
query = """
MATCH (g:Gwas)-[tophits:OPENGWAS_TOPHITS]->(v:Variant)-[]->(gene:Gene)-[gene_to_tissue:EXPRESSED_IN]->(tissue:Tissue)
where g.id = 'met-a-362'
AND gene_to_tissue.tpm > 1
WITH  v, gene_to_tissue, gene, tissue
ORDER BY gene_to_tissue.tpm  desc, tissue.name
RETURN gene.name, count(tissue.name) as N_tissues, 
collect(tissue.name) as Tissues, collect(gene_to_tissue.tpm) as TPM_vals
"""
df = query_to_df(session, query, True)
df



MATCH (g:Gwas)-[tophits:OPENGWAS_TOPHITS]->(v:Variant)-[]->(gene:Gene)-[gene_to_tissue:EXPRESSED_IN]->(tissue:Tissue)
where g.id = 'met-a-362'
AND gene_to_tissue.tpm > 1
WITH  v, gene_to_tissue, gene, tissue
ORDER BY gene_to_tissue.tpm  desc, tissue.name
RETURN gene.name, count(tissue.name) as N_tissues, 
collect(tissue.name) as Tissues, collect(gene_to_tissue.tpm) as TPM_vals



,gene.name,N_tissues,Tissues,TPM_vals
0,CPS1,39,"[Liver, Small Intestine - Terminal Ileum, Test...","[352.4, 36.46, 12.21, 9.24, 5.7395, 4.402, 2.9..."
1,CBS,51,"[Liver, Brain - Cerebellum, Brain - Cerebellar...","[135.9, 105.2, 87.155, 82.42, 72.08, 59.47, 52..."
2,DGKG,47,"[Brain - Cerebellar Hemisphere, Brain - Cerebe...","[60.23, 56.79, 42.365, 34.22, 11.47, 10.23, 10..."
3,ARHGEF11,52,"[Testis, Brain - Cerebellum, Brain - Cerebella...","[52.3, 49.61, 45.035, 31.66, 30.69, 30.56, 27...."
4,DMGDH,33,"[Liver, Kidney - Cortex, Adrenal Gland, Nerve ...","[40.8, 17.9, 4.8125, 4.811, 4.5895, 3.8965, 3...."
5,ZNF687,52,"[Testis, Bladder, Cells - EBV-transformed lymp...","[28.23, 23.37, 23.37, 23.15, 22.845, 22.49, 22..."
6,SLC6A12,44,"[Kidney - Cortex, Kidney - Cortex, Brain - Put...","[25.4, 25.4, 18.53, 18.53, 16.57, 16.57, 16.24..."


In [33]:
#  expression in breast tissue (probably does not make sense? )
query = """
MATCH (g:Gwas)-[tophits:OPENGWAS_TOPHITS]->(v:Variant)-[]->(gene:Gene)-[gene_to_tissue:EXPRESSED_IN]->(tissue:Tissue)
where g.id = 'met-a-362'
AND  tissue.name = "Breast - Mammary Tissue"
return distinct collect(v.name) as variants,  g.id,  gene.name,   gene_to_tissue.tpm as TPM, tissue.name
ORDER BY TPM desc
"""
df = query_to_df(session, query, True)
df


MATCH (g:Gwas)-[tophits:OPENGWAS_TOPHITS]->(v:Variant)-[]->(gene:Gene)-[gene_to_tissue:EXPRESSED_IN]->(tissue:Tissue)
where g.id = 'met-a-362'
AND  tissue.name = "Breast - Mammary Tissue"
return distinct collect(v.name) as variants,  g.id,  gene.name,   gene_to_tissue.tpm as TPM, tissue.name
ORDER BY TPM desc



,variants,g.id,gene.name,TPM,tissue.name
0,[rs2851391],met-a-362,ARHGEF11,18.85500,Breast - Mammary Tissue
1,[rs2851391],met-a-362,ZNF687,14.26000,Breast - Mammary Tissue
2,[rs2851391],met-a-362,CBS,6.28350,Breast - Mammary Tissue
3,[rs16876394],met-a-362,DMGDH,2.46300,Breast - Mammary Tissue
4,[rs715],met-a-362,CPS1,2.10150,Breast - Mammary Tissue
5,[rs2851391],met-a-362,DGKG,1.85550,Breast - Mammary Tissue
6,"[rs499368, rs499368]",met-a-362,SLC6A12,0.38425,Breast - Mammary Tissue


In [37]:
query = """
MATCH (n:Pathway)-[]-(p:Protein)-[]-(g:Gene)-[gene_to_tissue:EXPRESSED_IN]->(tissue:Tissue)
where n.id = 'R-HSA-6798163' //chlorine catabolism
AND gene_to_tissue.tpm > 1
WITH   gene_to_tissue, g, tissue
ORDER BY gene_to_tissue.tpm  desc, tissue.name
RETURN g.name, count(tissue.name) as N_tissues, 
collect(tissue.name) as Tissues, collect(gene_to_tissue.tpm) as TPM_vals
"""
df = query_to_df(session, query, True)
df


MATCH (n:Pathway)-[]-(p:Protein)-[]-(g:Gene)-[gene_to_tissue:EXPRESSED_IN]->(tissue:Tissue)
where n.id = 'R-HSA-6798163' //chlorine catabolism
AND gene_to_tissue.tpm > 1
WITH   gene_to_tissue, g, tissue
ORDER BY gene_to_tissue.tpm  desc, tissue.name
RETURN g.name, count(tissue.name) as N_tissues, 
collect(tissue.name) as Tissues, collect(gene_to_tissue.tpm) as TPM_vals



,g.name,N_tissues,Tissues,TPM_vals
0,BHMT,9,"[Liver, Kidney - Cortex, Bladder, Esophagus - ...","[287.6, 105.0, 18.46, 4.9030000000000005, 2.43..."
1,SLC44A1,52,"[Brain - Spinal cord (cervical c-1), Brain - S...","[109.2, 34.925, 30.755, 29.065, 28.08, 25.995,..."
2,ALDH7A1,51,"[Ovary, Brain - Amygdala, Brain - Caudate (bas...","[60.24, 53.625, 51.32, 45.83, 41.31, 40.5, 40...."
3,SARDH,32,"[Liver, Pancreas, Kidney - Cortex, Pituitary, ...","[41.64, 11.7, 3.7880000000000003, 3.728, 3.66,..."
4,DMGDH,33,"[Liver, Kidney - Cortex, Adrenal Gland, Nerve ...","[40.8, 17.9, 4.8125, 4.811, 4.5895, 3.8965, 3...."
5,CHDH,45,"[Liver, Kidney - Cortex, Minor Salivary Gland,...","[25.77, 19.99, 18.84, 17.58, 17.04, 16.9, 16.5..."


In [41]:
# other pathways that genes from gwas are involved in?
query = """
MATCH (g:Gwas)-[tophits:OPENGWAS_TOPHITS]->(v:Variant)-[]->(gene:Gene)-[]-(prot:Protein)-[inpath:PROTEIN_IN_PATHWAY]-(path:Pathway)
where g.id = 'met-a-362'
return distinct collect(gene.name) as genes,  path.name
"""
df = query_to_df(session, query, True)
df


MATCH (g:Gwas)-[tophits:OPENGWAS_TOPHITS]->(v:Variant)-[]->(gene:Gene)-[]-(prot:Protein)-[inpath:PROTEIN_IN_PATHWAY]-(path:Pathway)
where g.id = 'met-a-362'
return distinct collect(gene.name) as genes,  path.name



,genes,path.name
0,[DGKG],Effects of PIP2 hydrolysis
1,[DGKG],G alpha (q) signalling events
2,"[DGKG, ARHGEF11]",GPCR downstream signalling
3,[DGKG],Hemostasis
4,[DGKG],"Platelet activation, signaling and aggregation"
5,"[DGKG, ARHGEF11]",Signal Transduction
6,"[DGKG, ARHGEF11]",Signaling by GPCR
7,"[SLC6A12, SLC6A12]",Amino acid transport across the plasma membrane
8,"[SLC6A12, SLC6A12]",Creatine metabolism
9,"[SLC6A12, SLC6A12]","GABA synthesis, release, reuptake and degradation"
